Extreme Gradient Boosting (XGBoost)

In [2]:
pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 12.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.2.1 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [94]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.stats import randint, uniform, loguniform
from sklearn.model_selection import GridSearchCV

In [43]:
df = pd.read_csv('raw_data/archive/KAGGLE/DATASET-balanced.csv')

y= df[['LABEL']]
X= df.drop('LABEL', axis=1)
X

,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,0.338055,0.027948,2842.948867,4322.916759,6570.586186,0.041050,-462.169586,90.311272,19.073769,24.046888,...,-14.181895,-6.686564,0.902086,-7.251551,-1.198342,4.747403,-4.986279,0.953935,-5.013138,-6.779060
1,0.443766,0.037838,2336.129597,3445.777044,3764.949874,0.047730,-409.413422,120.348808,-7.161531,5.114784,...,0.372541,-2.131157,-6.876417,-1.359395,0.326401,-5.420016,-2.109968,-1.757634,-9.537907,-8.494421
2,0.302528,0.056578,2692.988386,2861.133180,4716.610271,0.080342,-318.996033,120.490273,-24.625771,23.891073,...,-3.099179,-5.853725,-3.724773,-6.627182,-5.117002,-6.072106,-0.994653,-1.617120,-3.922354,-7.033001
3,0.319933,0.031504,2241.665382,3503.766175,3798.641521,0.047180,-404.636749,136.320908,2.308172,-3.907071,...,1.513633,-1.898315,-2.046493,-7.176277,-3.293508,4.209121,0.121835,-5.407063,-3.654926,-3.274857
4,0.420055,0.016158,2526.069123,3102.659519,5025.077899,0.051905,-410.497925,152.731400,-18.266771,51.993462,...,11.086248,-1.952340,0.810868,6.238493,6.555839,7.535542,2.849219,2.616843,-1.793357,-5.060998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11773,0.435426,0.025303,2772.575031,2728.757601,4998.670213,0.074323,-342.309753,144.490418,-79.272942,8.890874,...,-2.950688,-17.982819,-7.831161,-1.127167,-7.669674,-0.653850,-8.037575,-2.671002,-4.483765,-3.355975
11774,0.454611,0.070578,1029.274601,1519.231563,1922.927486,0.026553,-332.230408,202.603012,-0.181929,-2.146542,...,-2.910435,-2.018668,-2.705635,-1.589172,-2.938737,-0.972690,-1.706672,-2.796168,2.171270,-1.660128
11775,0.374432,0.019063,4063.645317,3558.261357,7299.133512,0.110278,-372.149109,92.670235,-29.082432,59.736637,...,-17.552000,-6.628118,-3.827499,-7.287946,-2.899543,-11.508186,-1.296590,-14.325416,-4.405540,-15.869982
11776,0.410885,0.090499,1124.655596,1553.651133,2065.942806,0.031761,-328.062805,193.557526,6.779151,-1.304731,...,-7.348275,-5.437202,-4.252508,-1.258683,-2.107233,-1.018154,-2.716950,-3.681598,3.811063,3.948419


In [46]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y=le.fit_transform(y)
y

/Users/andimand/.pyenv/versions/3.10.6/envs/Deep_fake_voice_recognition/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 0, 0, ..., 1, 1, 1])

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

grid = {'n_estimators': randint(1, 400),
        'max_depth': randint(1, 13),  
        'learning_rate': loguniform (0.001, 10)
       }
bst = XGBClassifier()
# Instantiate Grid Search
search = RandomizedSearchCV(
    bst,
    grid, 
    scoring='accuracy',
    n_iter=1000,  
    cv=5, n_jobs=-1
)

search.fit(X_train, y_train)
search.best_estimator_

python(63525) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63526) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63527) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63528) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63529) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63530) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63531) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63532) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.22311250284899017,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=347, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [96]:
grid = {'n_estimators': range(320, 360),
        'max_depth': range(4,7),  
        'learning_rate': [0.2, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28]
       }
bst = XGBClassifier()
# Instantiate Grid Search
search = GridSearchCV(
    bst,
    grid, 
    scoring='accuracy',
    cv=5, n_jobs=-1
)
search.fit(X_train, y_train)
search.best_estimator_

python(63654) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63655) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63656) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63657) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63658) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63659) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63660) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63661) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.22, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=329, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
bst = XGBClassifier(n_estimators=347, max_depth=6, learning_rate=0.22311250284899017, objective='binary:logistic')

0.9924434519625549

In [79]:
bst.fit(X_train, y_train)
preds = bst.predict(X_test)
preds

array([1, 0, 0, ..., 0, 1, 0])

In [87]:

# Evaluating accuracy
accuracy = accuracy_score(y_test, preds)
print(f"Accuracy: {accuracy * 100.0}%")
print('Precision =', round(precision_score(y_test, preds), 6)) # Precision

print('Recall =', round(recall_score(y_test, preds), 6)) # Recall

print('F1 score =', round(f1_score(y_test, preds), 6)) # F1 score

Accuracy: 99.29258630447086%
Precision = 0.992533
Recall = 0.993103
F1 score = 0.992818


In [112]:


from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100)
cv_results = cross_validate(forest, X_train, y_train, scoring = "accuracy", cv=5)


In [117]:
cv_results['test_score'].mean()

0.984109154121062

In [118]:
grid = {'n_estimators': range(0,100)}

search = GridSearchCV(
    forest,
    grid, 
    scoring='accuracy',
    cv=5, n_jobs=-1
)

In [119]:
search.fit(X_train, y_train)
search.best_estimator_

python(64413) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(64414) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(64415) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(64417) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(64418) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(64419) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(64420) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(64421) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
/Users/andimand/.pyenv/versions/3.10.6/envs/Deep_fake_voice_recognition/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
5 fits failed out of a total of 500.
The score on th

RandomForestClassifier(n_estimators=99)

In [125]:
forest = RandomForestClassifier(n_estimators=99)

In [128]:
forest.fit(X_train, y_train)
pred_forest = forest.predict(X_test)

accuracy = accuracy_score(y_test, pred_forest)
print(f"Accuracy: {accuracy * 100.0}%")
print('Precision =', round(precision_score(y_test, pred_forest), 6)) # Precision

print('Recall =', round(recall_score(y_test, pred_forest), 6)) # Recall

print('F1 score =', round(f1_score(y_test, pred_forest), 6)) # F1 score


Accuracy: 98.64176570458405%
Precision = 0.990151
Recall = 0.982184
F1 score = 0.986151


In [129]:
import librosa
import soundfile as sf

# Load the audio file
filename = 'raw_data/archive/KAGGLE/AUDIO/Ryan Gosling interview.wav'
y, sr = librosa.load(filename, sr=None)

# Define start and end times
start_sec = 2 * 60  # 2 minutes
end_sec = start_sec + 4  # 4 seconds after the start

# Convert times to samples
start_sample = int(start_sec * sr)
end_sample = int(end_sec * sr)

# Extract the segment
segment = y[start_sample:end_sample]

# Save the segment
out_filename = 'extracted_segment_ryan.wav'
sf.write(out_filename, segment, sr)



python(65973) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(65974) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
